An analysis on creating the joint positions / time for our quad wave gait.

# Highest Imports

Set up matplotlib

In [5]:
!pip3 install ipympl

  Using cached https://files.pythonhosted.org/packages/d9/6c/5f4b4764d3009027b4a556aebb35a09d09c09e1996ed701a150172790d98/ipympl-0.7.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 122kB 2.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/11/53/084940a83a8158364e630a664a30b03068c25ab75243224d6b488800d43a/ipywidgets-7.6.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/03/b7b30fa81cb687d1178e085d0f01111ceaea3bf81f9330c937fb6f6c8ca0/matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 430kB 1.5MB/s ta 0:00:01
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 788kB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 4.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ca/ab/872a23e29cec3cf2594af7e857f18b687ad21039c1f9b922fac5b9b142d5/traitlets-4.3.3-py2.py3-none-any.whl
    100% |█████████████

  Using cached https://files.pythonhosted.org/packages/c5/8f/51e89ce52a085483359217bc72cdbf6e75ee595d5b1d4b5ade40c7e018b8/jsonschema-3.2.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 9.5MB 120kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/22/a6/858897256d0deac81a172289110f31629fc4cee19b6f01283303e18c8db3/ptyprocess-0.7.0-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 102kB 7.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/59/7c/e39aca596badaf1b78e8f547c807b04dae603a433d3e7a7e04d67f2ef3e5/wcwidth-0.2.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/aa/cb45262569fcc047bf070b5de61813724d6726db83259222cd7b4c79821a/attrs-20.3.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 112kB 7.8MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 6.7MB/s ta 0:00:01
  Using cached https://files.pythonho

  Using cached https://files.pythonhosted.org/packages/3e/89/7ea760b4daa42653ece2380531c90f64788d979110a2ab51049d92f408af/packaging-20.9-py2.py3-none-any.whl
  Running setup.py bdist_wheel for pyrsistent ... done
  Stored in directory: /home/revant/.cache/pip/wheels/f0/b8/de/b593ad311be4eb458499d100db081e453576032272398b7ddc
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /home/revant/.cache/pip/wheels/93/9a/79/b2c3567908fd6209e4674ca23d9fcf005aae5fe89148913727
Successfully built pyrsistent pandocfilters


In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ipympl'

Other useful imports

In [2]:
import numpy as np

# Gait Parameters

In [3]:
beta = duty_cycle = 0.75
v = body_velocity = 0.03  # m/s
T = cycle_time = 1  # s
L = v * T # m

Compute $T_l$, the relative cycle time for each leg

In [4]:
T_l = (1 - beta) * T

Vertical Leg movement parameters

In [5]:
# Maximum distance to raise the foot tip
y_max = .015  #m

# Generate Horizontal + Vertical Positions & Velocites

We will split this up into 6 checkpoints, similar to how it was done in the video. 
1. vertical launch (when the leg is lifting)
2. horizontal launch (leg is done lifting up and starts moving horizontally)
3. horizontal movement (leg is moving the final position)
4. horizontal movement stop
5. horizontal stop
6. vertical stop

Observe that the horizontal velocity graph is always symmetric, so we can split our checkpoints into the two following groups:
1. horizontal_move_start
2. horizontal_move_stable

As the leg always needs to move first, we know that $launch_{vertical} = 0$ always. Keep them as fractions of the **half cycle** as everything is symmetric and dynamic

In [6]:
horizontal_move_start = .4
horizontal_move_stable = .8

checkpoint_fracs = np.array((0, horizontal_move_start, horizontal_move_stable))
left = checkpoint_fracs * T_l / 2
right = T_l - left 
t = np.concatenate((left, np.flip(right)))
t

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25])

Visualize the checkpoints

In [7]:
plt.figure()
plt.scatter(t, np.full(t.shape, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Horizontal Velocity Over Time
Compute the x-velocities for all timesteps

In [8]:
vx_max = L / ((t[2] - t[1]) + (t[3] - t[2]))
vx = np.array((0, 0, vx_max, vx_max, 0, 0))

In [9]:
L

0.03

Visualize

In [10]:
plt.figure()
plt.plot(t, vx)
plt.title('$v_x$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$v_x$ vs $t$')

## Horizontal Displacement over time
Recall that position ($x$) can be found as

$$x(t + \Delta T) = x(t) + \left. \int vx dx \right]_{t-1}^{t}$$

In [11]:
def vel_to_pos(start, t, vels):
    d_curr = start
    d = [d_curr]
    for i in range(1, len(t)):        
        if vels[i] == vels[i - 1]:  # Square integration
            d_curr += (t[i] - t[i - 1]) * vels[i]
        else:
            if vels[i] == 0:
                sgn = np.sign(vels[i - 1])
            else:
                sgn = np.sign(vels[i])
            
            # Compute square area
            square = (t[i] - t[i - 1]) * min(abs(vels[i]), abs(vels[i - 1]))
            
            # Compute triangle area
            triangle = .5 * (t[i] - t[i - 1]) * abs(vels[i] - vels[i - 1]) * sgn
            
            d_curr += (square + triangle)
        d.append(d_curr)
    return np.array(d)

Recall that the position graph can be found for

In [12]:
x_0 = -L / 2
x = vel_to_pos(x_0, t, vx)

In [13]:
t

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25])

Visualize

In [14]:
plt.figure()
plt.plot(t, x)
plt.title('$x$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$x$ vs $t$')

## Vertical Velocity over Time

In [15]:
vy_max = 2 * y_max / (t[2] - t[0]) 
vy = np.array([0, vy_max, 0, 0, -vy_max, 0])

Visualize

In [16]:
plt.figure()
plt.plot(t, vy)
plt.title('$vy$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$vy$ vs $t$')

## Vertical Displacement Over Time

In [17]:
y_0 = 0
y = vel_to_pos(y_0, t, vy)

Visualize

In [18]:
plt.figure()
plt.plot(t, y)
plt.title('$y$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $t$')

## Vertical Vel vs. Horizontal Vel w.r.t Ground

In [19]:
plt.figure()
plt.plot(x, y)
plt.title('$y$ vs $x$ w.r.t ground')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $x$ w.r.t ground')

## Vertical Positiosn vs. Horizontal positions w.r.t Body

In [20]:
x_b = x - v * t

Visualize

In [21]:
plt.figure()
plt.plot(x_b, y)
plt.title('$y$ vs $x$ w.r.t body')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $x$ w.r.t body')